# Fashion Retail Sales Dataset

In [1]:
import pandas as pd

df = pd.read_csv('Fashion_Retail_Sales.csv')

In [2]:
df.shape

(3400, 6)

In [3]:
df.columns

Index(['Customer Reference ID', 'Item Purchased', 'Purchase Amount (USD)',
       'Date Purchase', 'Review Rating', 'Payment Method'],
      dtype='object')

In [4]:
df.rename(columns={
    'Customer Reference ID':'customer_id',
    'Item Purchased': 'item_purchased',
    'Purchase Amount (USD)': 'purchase_amount',
    'Date Purchase': 'purchase_date',
    'Review Rating': 'review_rating',
    'Payment Method': 'payment_method',
}, inplace=True)

In [5]:
df.describe(include='all')

,customer_id,item_purchased,purchase_amount,purchase_date,review_rating,payment_method
count,3400.000000,3400,2750.000000,3400,3076.000000,3400
unique,NaN,50,NaN,365,NaN,2
top,NaN,Belt,NaN,22-09-2023,NaN,Credit Card
freq,NaN,90,NaN,17,NaN,1770
mean,4039.660588,NaN,156.709818,NaN,2.999057,NaN
std,48.122583,NaN,419.536669,NaN,1.156505,NaN
min,3957.000000,NaN,10.000000,NaN,1.000000,NaN
25%,3997.000000,NaN,57.000000,NaN,2.000000,NaN
50%,4040.000000,NaN,110.000000,NaN,3.000000,NaN
75%,4081.000000,NaN,155.750000,NaN,4.000000,NaN


In [6]:
# check percentage of missing values
missing_values = df.isnull().mean() * 100
missing_values[missing_values > 0].sort_values(ascending=False)

purchase_amount    19.117647
review_rating       9.529412
dtype: float64

In [7]:
# check which columns have missing values
missing_columns = df.columns[df.isnull().any()].tolist()
missing_columns

['purchase_amount', 'review_rating']

In [8]:
# since the price is dependent on the 'item_purchased'. A much better approach is to group the data by 'item_purchased', 
# calculate the median price for each specific item, and then fill the missing values for an item with its own median price.
df['purchase_amount'].fillna(df.groupby('item_purchased')['purchase_amount'].transform('median'), inplace=True)

C:\Users\kweku\AppData\Local\Temp\ipykernel_4740\2179996.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['purchase_amount'].fillna(df.groupby('item_purchased')['purchase_amount'].transform('median'), inplace=True)


In [9]:
# Just like with price, it's more accurate to fill the missing rating for an item with the median rating of that specific item.
df['review_rating'].fillna(df.groupby('item_purchased')['review_rating'].transform('median'), inplace=True)

C:\Users\kweku\AppData\Local\Temp\ipykernel_4740\932829687.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['review_rating'].fillna(df.groupby('item_purchased')['review_rating'].transform('median'), inplace=True)


In [10]:
# convert 'purchase_date' to datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='%d-%m-%Y')

In [11]:
# feature engineering: create new, useful columns from the 'purchase_date' column, such as 'month', 'year', and 'day_of_week'
df['month'] = df['purchase_date'].dt.month
df['year'] = df['purchase_date'].dt.year
df['day_of_week'] = df['purchase_date'].dt.day_name()

In [12]:
# save the cleaned DataFrame to a new CSV file
df.to_csv('fashion_retail_sales_cleaned.csv', index=False)

In [13]:
# Top 10 best-selling items by quantity
top_items = df['item_purchased'].value_counts().head(10)

# Top 10 items by total revenue
top_revenue_items = df.groupby('item_purchased')[
    'purchase_amount'].sum().sort_values(ascending=False).head(10)

# Most common payment method
top_payment = df['payment_method'].value_counts()

# Daily sales trend
daily_sales = df.groupby('purchase_date')['purchase_amount'].sum()

# Monthly sales trend
monthly_sales = df.groupby(['year', 'month'])['purchase_amount'].sum().reset_index()

# Yearly sales trend
yearly_sales = df.groupby('year')['purchase_amount'].sum().reset_index()

# PRINTING RESULTS
print("Top 10 Best-Selling Items by Quantity:")
print(top_items)
print("\nTop 10 Items by Total Revenue:")
print(top_revenue_items)
print("\nMost Common Payment Method:")
print(top_payment)
print("\nDaily Sales Trend:")
print(daily_sales)
print("\nMonthly Sales Trend:")
print(monthly_sales)
print("\nYearly Sales Trend:")
print(yearly_sales)


Top 10 Best-Selling Items by Quantity:
item_purchased
Belt        90
Skirt       88
Shorts      87
Pants       86
Tank Top    82
T-shirt     82
Pajamas     81
Camisole    76
Loafers     76
Hoodie      75
Name: count, dtype: int64

Top 10 Items by Total Revenue:
item_purchased
Tunic       18143.0
Pajamas     15178.0
Jeans       14378.0
Shorts      14138.5
Handbag     13890.0
Gloves      13142.0
Boots       13087.0
Loafers     13037.0
Poncho      12871.5
Slippers    12730.0
Name: purchase_amount, dtype: float64

Most Common Payment Method:
payment_method
Credit Card    1770
Cash           1630
Name: count, dtype: int64

Daily Sales Trend:
purchase_date
2022-10-02    1056.0
2022-10-03    1293.0
2022-10-04    1161.5
2022-10-05    1375.5
2022-10-06     766.5
               ...  
2023-09-27     546.0
2023-09-28    1341.0
2023-09-29     604.0
2023-09-30    1098.5
2023-10-01     423.0
Name: purchase_amount, Length: 365, dtype: float64

Monthly Sales Trend:
    year  month  purchase_amount
0   